# Problem Set #5

## Team exercises

---

_Student ID and Name_:

_Student ID and Name_:

_Student ID and Name_:

---

### Objective

During this activity, students should be able to:

* Modify the Lisp metacircular evaluator in order to extend its functionality. 

---

### Instructions

- With all the team members of your breakout room, modify the Lisp metacircular evaluator built in class in order to solve the following problems. Make sure each function passes all the unit tests.

- Once the team has finished all the problems, use [this link](http://34.212.143.74/apps/s202013/tc2006/programming_problem_set_5/) to upload the notebook file.

- Only one team member needs to upload the file.

- Due date is Monday, Ocotber 5.

# Lisp Interpreter

## Helper functions

In [1]:
(defn third
  [lst]
  (nth lst 2))

#'user/third

In [2]:
(defn fourth
  [lst]
  (nth lst 3))

#'user/fourth

## Closure Data Structure

In [3]:
(import 'clojure.lang.IFn)

(declare $eval)

(deftype Closure
  [env params body]
  
  IFn
  
  (applyTo [self args]
    (let [extended-env (merge @env (zipmap params args))]
      ($eval body extended-env))))

(defn make-closure
  [env params body]
  (->Closure (atom env) params body))

(defn set-closure-env!
  [label-name closure]
  (swap! (.env closure)
         #(assoc % label-name closure)))

#'user/set-closure-env!

## Main interpreter function

In [4]:
(defn $eval
  [expr env]
  
  (cond
    
    ; Check for variable reference
    (symbol? expr)
    (if (contains? env expr)
      (env expr)
      (throw (RuntimeException. (str "Unbound variable: " expr))))
    
    ; Check for special forms
    (list? expr)
    (case (first expr)
      
      nil
      ()
      
      QUOTE
      (second expr)
      
      IF
      (let [condition (second expr)
            then (third expr)
            else (fourth expr)]
        (if ($eval condition env)
          ($eval then env)
          ($eval else env)))
      
      LAMBDA
      (let [params (second expr)
            body (third expr)]
        (make-closure env params body))
      
      LABEL
      (let [label-name (second expr)
            closure ($eval (third expr) env)]
        (set-closure-env! label-name closure)
        closure)
      
      DO
      (loop [x ($eval (second expr) env)
             lst (rest (rest expr))]
        (if (empty? lst)
          x
          (recur ($eval (first lst) env)
                 (rest lst))))
      
      DOTIMES
      (let [var   (first (second expr))
            count ($eval (second (second expr)) env)
            body  (third expr)]
        (doseq [i (range count)]
          ($eval body (assoc env var i))))
      
      ; Ordinary function application
      (let [eval-all-expr (map #($eval % env) expr)
            fun (first eval-all-expr)
            args (rest eval-all-expr)]
        (apply fun args)))
    
    ; Anything that is not symbol or a list evals to itself
    :else
    expr))

#'user/$eval

## Unit Tests

In [5]:
(require '[clojure.test :refer [is]])

nil

In [6]:
(is (= 42 ($eval 42 {})))
(is (= true ($eval true {})))
(is (= "hello" ($eval "hello" {})))

true

In [7]:
(is (= 5 ($eval 'X {'X 5})))
(is (= 7 ($eval 'C {'A 5, 'B 6, 'C 7})))

true

In [8]:
(is (= 'A ($eval '(QUOTE A) {})))
(is (= '(1 2 3) ($eval '(QUOTE (1 2 3)) {})))
(is (= 42 ($eval '(QUOTE 42) {})))

true

In [9]:
(is (= 2 ($eval '(IF 1 2 3) {})))
(is (= 3 ($eval '(IF nil 2 3) {})))

true

In [10]:
(is (= 2 ($eval '(ADD 1 1) {'ADD +})))
(is (= '(1 2 3 4)  ($eval '(CONS 1 (QUOTE (2 3 4))) {'CONS cons})))
(is (= 1 ($eval '(CAR X) {'CAR first, 'X '(1 2 3 4)})))

true

In [11]:
(is (= 15 ($eval '((LAMBDA (X Y) 
                     (* X (+ Y 1))) 
                   3
                   4) 
                 {'+ +, '* *})))
(is (= Closure (type ($eval '(LAMBDA (X) X) {}))))
(is (= 42 ($eval '((LAMBDA () 42)) {})))
(is (= 8 ($eval '((LAMBDA (F X) 
                    (F (F (F X))))
                  (LAMBDA (X) (* X 2))
                  1)
                {'* *})))

true

In [12]:
(is (= '(A A B B C C D D) 
       ($eval '((LABEL DUP (LAMBDA (X)
                            (IF (NULL? X)
                                ()
                                (CONS (CAR X)
                                      (CONS (CAR X)
                                            (DUP (CDR X)))))))
                (QUOTE (A B C D))) 
              {'NULL? empty?,
               'CONS cons,
               'CAR first,
               'CDR rest})))
(is (= 120
       ($eval '((LABEL FACT (LAMBDA (N)
                              (IF (ZERO? N)
                                1
                                (* N (FACT (- N 1))))))
                5) 
              {'ZERO? zero?,
               '* *,
               '- -})))

true

In [13]:
;;; Unit tests:
(is (nil? ($eval '(DO)
                 {})))
(is (= 3
       ($eval '(DO (+ 1 2))
              {'+ +})))
(is (= 10
       ($eval '(DO (+ 2 5)
                   (- 2 5)
                   (/ 2 5)
                   (* 2 5))
              {'+ +
               '- -
               '/ /
               '* *})))
(is (= "7-32/510"
       (with-out-str
         ($eval '(DO (PR (+ 2 5))
                     (PR (- 2 5))
                     (PR (/ 2 5))
                     (PR (* 2 5)))
                {'PR pr
                 '+ +
                 '- -
                 '/ /
                 '* *}))))
(is (= (let [nl (System/lineSeparator)]
         (str "1" nl "2" nl "3" nl))
       (with-out-str
         ($eval '(DO (PRN (+ -2 (+ 1 2)))
                     (PRN (+ 1 1))
                     (PRN 3)
                     (+ 2 2))
                {'+ +
                 'PRN prn}))))
(is (= (let [nl (System/lineSeparator)]
         (str "One potato," nl
              "two potatoes," nl
              "three potatoes," nl
              "four." nl))
       (with-out-str
         ($eval '(DO (PRINTLN "One potato,")
                     (PRINTLN "two potatoes,")
                     (PRINTLN "three potatoes,")
                     (PRINTLN "four."))
                {'PRINTLN println}))))

true

In [14]:
;;; Unit tests:
(is (= ""
       (with-out-str ($eval '(DOTIMES (x 0)
                               (PRINTLN x))
                            {'PRINTLN println}))))
(is (= "0123456789"
       (with-out-str ($eval '(DOTIMES (x 10)
                               (PR x))
                            {'PR pr}))))
(is (= (let [nl (System/lineSeparator)]
         (str "Line 0" nl "Line 1" nl "Line 2" nl "Line 3" nl))
       (with-out-str ($eval '(DOTIMES (i (+ 2 2))
                               (PRINTLN "Line" i))
                            {'PRINTLN println, '+ +}))))
(is (= "1-4-9-16-25-36-49-64-81-100-"
       (with-out-str ($eval '(DOTIMES (some-var (* 2 5))
                               (PRINTF "%d-"
                                       ((LAMBDA (x) (* x x))
                                        (INC some-var))))
                            {'PRINTF printf, '* *, 'INC inc}))))
(is (= (str "**************************************************"
            "**************************************************")
       (with-out-str ($eval '(DOTIMES (mxyzptlk (* 2 2 5 5))
                               (PRINT "*"))
                            {'PRINT print, '* *}))))

true